# 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import math
from tqdm import tqdm
import torch
from torch import nn, optim                           
from torch.optim.lr_scheduler import _LRScheduler
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import cKDTree
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.metrics.pairwise import haversine_distances

# 랜덤 시드 설정

In [133]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 데이터 불러오기

In [134]:
data = pd.read_csv('../data/merged_data.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')

In [135]:
# 시간 순으로 정렬
data = data.sort_values(['contract_year_month', 'contract_day'], ascending= False)
data['deposit_per_area'] = data['deposit'] / data['area_m2']

In [137]:
train_data = data[data._type == 'train'].drop('_type', axis=1)
test_data = data[data._type == 'test'].drop('_type', axis=1)

# 각 아파트별 최신 20개 거래만 필터링
train_data['cum_count'] = train_data.groupby(['latitude', 'longitude']).cumcount()
train_recent_data = train_data[train_data.cum_count < 20].reset_index(drop=True)

In [140]:
# train data에 등장하는 아파트별 변수 생성
deposit_mean = train_recent_data.groupby(['latitude', 'longitude'])['deposit_per_area'].mean().reset_index()
background = train_recent_data[['latitude', 'longitude', 'park_count_1000m', 'school_count_within_1km', 'subway_count_1km']].drop_duplicates().reset_index(drop=True)
floor_max  = data.groupby(['latitude', 'longitude']).agg({'floor' : 'max', 'built_year' : 'min'}).reset_index()

train_apt = deposit_mean.merge(background, how= 'left')
train_apt = train_apt.merge(floor_max, how= 'left')

In [142]:
# test data에 새롭게 등장하는 아파트 데이터 생성
te_df = test_data[['latitude', 'longitude']].drop_duplicates()
tr_df = train_data[['latitude', 'longitude']].drop_duplicates()
new_apt = pd.concat([te_df, tr_df, tr_df]).drop_duplicates(keep=False).reset_index(drop=True)
new_apt['_type'] = 'test'

test_data2 = test_data.merge(new_apt, how='inner')
test_apt = test_data2.groupby(['latitude','longitude','park_count_1000m', 'school_count_within_1km', 'subway_count_1km']).agg({'floor':'max', 'built_year' : 'min'}).reset_index()

In [183]:
# train, test 아파트 데이터 합치기
data_apt = pd.concat([train_apt, test_apt]).reset_index(drop=True)
data_apt

,latitude,longitude,deposit_per_area,park_count_1000m,school_count_within_1km,subway_count_1km,floor,built_year
0,36.917910,126.908029,101.734771,2,0,0,14,1996
1,36.957089,127.047449,143.765026,12,1,0,5,1990
2,36.959894,127.045371,232.572587,10,0,0,12,2020
3,36.960034,127.059939,200.489081,13,4,0,13,2005
4,36.960936,127.054219,185.547771,13,4,0,8,2010
...,...,...,...,...,...,...,...,...
18671,37.819235,127.055680,NaN,8,5,1,22,2024
18672,37.828185,127.076071,NaN,17,3,0,27,2024
18673,37.835581,127.084910,NaN,12,2,0,29,2024
18674,37.885999,127.196120,NaN,6,1,0,20,2024


# 그래프 데이터 만들기

In [185]:
train_graph = data_apt[~data_apt.deposit_per_area.isna()]
test_graph = data_apt[data_apt.deposit_per_area.isna()]

In [186]:
X_train = train_graph.drop(['latitude','longitude', 'deposit_per_area'], axis=1)
X_test = test_graph.drop(['latitude','longitude', 'deposit_per_area'], axis=1)

In [187]:
# 스케일링
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [188]:
X_train = pd.DataFrame(X_train_scaled, columns = X_train.columns).reset_index(drop=True)
X_test = pd.DataFrame(X_test_scaled, columns = X_test.columns).reset_index(drop=True)

In [189]:
train_graph = pd.concat([train_graph[['latitude', 'longitude', 'deposit_per_area']].reset_index(drop=True),X_train], axis=1)
test_graph = pd.concat([test_graph[['latitude', 'longitude', 'deposit_per_area']].reset_index(drop=True),X_test], axis=1)

In [191]:
graph_data = pd.concat([train_graph, test_graph], axis=0).reset_index(drop=True)
graph_data

,deposit_per_area,latitude,longitude,park_count_1000m,school_count_within_1km,subway_count_1km,floor,built_year
0,101.734771,0.000000,0.363158,0.040816,0.00,0.000000,0.217391,0.555556
1,143.765026,0.030995,0.481092,0.244898,0.04,0.000000,0.086957,0.460317
2,232.572587,0.033215,0.479334,0.204082,0.00,0.000000,0.188406,0.936508
3,200.489081,0.033325,0.491657,0.265306,0.16,0.000000,0.202899,0.698413
4,185.547771,0.034039,0.486818,0.265306,0.16,0.000000,0.130435,0.777778
...,...,...,...,...,...,...,...,...
18671,NaN,0.713059,0.488054,0.163265,0.20,0.166667,0.333333,1.000000
18672,NaN,0.720139,0.505302,0.346939,0.12,0.000000,0.405797,1.000000
18673,NaN,0.725990,0.512779,0.244898,0.08,0.000000,0.434783,1.000000
18674,NaN,0.765877,0.606851,0.122449,0.04,0.000000,0.304348,1.000000


In [195]:
def create_graph(df, distance_threshold=1):
    train_index = torch.tensor(range(0,15000))
    holdout_index = torch.tensor(range(15000,18491))
    test_index = torch.tensor(range(18491,18676))
    
    # 노드 특성 생성
    node_features = df.drop(['latitude', 'longitude', 'deposit_per_area'], axis=1).values

    df2 = df.copy()
    df2.latitude = df2.latitude * 111.139
    df2.longitude = df2.longitude * 111.321
    locations = np.array(df2[['latitude', 'longitude']].values)
    
    # KD-tree를 사용한 효율적인 거리 기반 엣지 생성
    tree = cKDTree(locations)
    edges = tree.query_pairs(r=distance_threshold, output_type='ndarray')
    edge_index = torch.tensor(edges.T, dtype=torch.long)
    
    # 타겟 값 (전세가)
    y = torch.tensor(df['deposit_per_area'].values, dtype=torch.float)

    # PyTorch Geometric 데이터 객체 생성
    data = Data(x=torch.tensor(node_features, dtype=torch.float),
                edge_index=edge_index,
                y=y)
    data.train_mask=train_index
    data.val_mask=holdout_index
    data.test_mask=test_index
    
    return data

In [196]:
dataset = create_graph(graph_data, distance_threshold=1)

In [197]:
dataset

Data(x=[18676, 5], edge_index=[2, 710499], y=[18676], train_mask=[15000], val_mask=[3491], test_mask=[185])

# GNN 모델 훈련

In [157]:
class GNNModel(torch.nn.Module):
    def __init__(self, input_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, 32)
        # self.conv2 = GCNConv(64, 32)
        self.conv3 = GCNConv(32, 16)
        self.linear = torch.nn.Linear(16, 1)

    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = torch.relu(self.conv1(x, edge_index))
        # x = torch.relu(self.conv2(x, edge_index))
        x = torch.relu(self.conv3(x, edge_index))
        x = self.linear(x)
        
        return x.squeeze()

# 4. 학습 함수
def train(model, data_loader, criterion, optimizer, epochs, device, checkpoint_name, scheduler = None):
    model.train()

    best_loss = np.inf
    for epoch in tqdm(range(epochs)):
        train_loss = 0
        optimizer.zero_grad()
        out = model(data_loader)
        loss = criterion(out[data_loader.train_mask], data_loader.y[data_loader.train_mask])
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_loss /= len(dataset.train_mask)
        valid_loss = evaluate(model, data_loader, criterion)
        
        if valid_loss < best_loss: 
            torch.save(model.state_dict(), f'checkpoint/{checkpoint_name}_parameters.pt')
            best_loss = valid_loss
        
        if scheduler:
            scheduler.step()
            
        print(f'Epoch: {epoch+1}, Training Loss: {train_loss}, Validation Loss: {valid_loss}') 



# 5. 평가 함수
def evaluate(model, data_loader, criterion):
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        out = model(data_loader)
        loss = criterion(out[data_loader.val_mask], data_loader.y[data_loader.val_mask])
        valid_loss += loss.item()
    return valid_loss / len(dataset.val_mask)

In [158]:
# schedular
class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]
    
    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr


In [121]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNModel(input_dim=dataset.num_features)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
optimizer = optim.Adam(model.parameters(), lr = 0)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=50, T_mult=1, eta_max=0.1,  T_up=10, gamma=0.5)
criterion = nn.L1Loss(reduction='sum')
checkpoint_name = 'GNN_baseline'

epochs = 200
train(model, dataset, criterion, optimizer, epochs, device, checkpoint_name, scheduler)

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 2/400 [00:00<02:06,  3.15it/s]

Epoch: 1, Training Loss: 477.2844333333333, Validation Loss: 362.66571183042106
Epoch: 2, Training Loss: 477.2844333333333, Validation Loss: 362.58926525350904


  1%|          | 3/400 [00:00<01:45,  3.77it/s]

Epoch: 3, Training Loss: 477.20326666666665, Validation Loss: 362.396197364652


...

Epoch: 397, Training Loss: 203.34031666666667, Validation Loss: 140.26814487252935


100%|█████████▉| 398/400 [01:31<00:00,  4.42it/s]

Epoch: 398, Training Loss: 203.34023333333334, Validation Loss: 140.26815382411917


100%|██████████| 400/400 [01:31<00:00,  4.35it/s]

Epoch: 399, Training Loss: 203.3402, Validation Loss: 140.26815382411917
Epoch: 400, Training Loss: 203.3402, Validation Loss: 140.26815382411917


In [122]:
gnn_model = GNNModel(dataset.num_features)
gnn_model.load_state_dict(torch.load(f'checkpoint/{checkpoint_name}_parameters.pt'))

<All keys matched successfully>

# Holdout 데이터셋에 대한 성능 확인

In [159]:
gnn_model.eval()
with torch.no_grad():
    out = gnn_model(dataset)
    
graph_data['pred'] = out

In [161]:
holdout_start = 202307
holdout_end = 202312
holdout_data = train_data[(train_data['contract_year_month'] >= holdout_start) & (train_data['contract_year_month'] <= holdout_end)]
train_data = train_data[~((train_data['contract_year_month'] >= holdout_start) & (train_data['contract_year_month'] <= holdout_end))]

In [162]:
holdout_data = holdout_data.merge(graph_data[['latitude', 'longitude', 'pred']], how='left')
holdout_data['deposit_pred'] = holdout_data['pred'] * holdout_data['area_m2']

In [199]:
# MAE
gnn_holdout_mae = sum(abs(holdout_data.deposit_pred - holdout_data.deposit))/len(holdout_data)
print("Holdout 데이터셋 성능:")
print(f"GNN MAE: {gnn_holdout_mae:.2f}")

Holdout 데이터셋 성능:
GNN MAE: 19782.30
